In [1]:
import json
import random
from datetime import datetime, timedelta

# Sample Inputs
venues = [
    {"id": 1, "name": "Hall A", "capacity": 50},
    {"id": 2, "name": "Lab B", "capacity": 30},
    {"id": 3, "name": "Auditorium", "capacity": 100},
    {"id": 4, "name": "Room C", "capacity": 40},
]

# Programs, Departments, and Courses
programs = {
    "College of Computing and Information Science": {
        "Software Engineering": [
            {"code": "SEN101", "name": "Introduction to Software", "level": 100, "students": 60, "type": "Written"},
            {"code": "SEN102", "name": "Programming Fundamentals", "level": 100, "students": 55, "type": "CBT"},
            {"code": "SEN201", "name": "Data Structures", "level": 200, "students": 50, "type": "Written"},
            {"code": "SEN202", "name": "Algorithms", "level": 200, "students": 45, "type": "CBT"},
            {"code": "SEN301", "name": "Software Architecture", "level": 300, "students": 40, "type": "Written"},
            {"code": "SEN302", "name": "Web Development", "level": 300, "students": 35, "type": "CBT"},
            {"code": "SEN401", "name": "AI Fundamentals", "level": 400, "students": 30, "type": "Written"},
            {"code": "SEN402", "name": "Machine Learning", "level": 400, "students": 25, "type": "CBT"},
        ],
        "Computer Science": [
            {"code": "CSC101", "name": "Computer Basics", "level": 100, "students": 60, "type": "Written"},
            {"code": "CSC102", "name": "Digital Logic", "level": 100, "students": 55, "type": "CBT"},
            {"code": "CSC201", "name": "Operating Systems", "level": 200, "students": 50, "type": "Written"},
            {"code": "CSC202", "name": "Database Systems", "level": 200, "students": 45, "type": "CBT"},
            {"code": "CSC301", "name": "Computer Networks", "level": 300, "students": 40, "type": "Written"},
            {"code": "CSC302", "name": "Cryptography", "level": 300, "students": 35, "type": "CBT"},
            {"code": "CSC401", "name": "Cybersecurity", "level": 400, "students": 30, "type": "Written"},
            {"code": "CSC402", "name": "Cloud Computing", "level": 400, "students": 25, "type": "CBT"},
        ],
    }
}

invigilators = [
    {"id": 1, "name": "Dr. Smith"},
    {"id": 2, "name": "Prof. Johnson"},
    {"id": 3, "name": "Dr. Lee"},
    {"id": 4, "name": "Prof. Carter"},
]

# Configurations
exam_slots = [("08:00", "10:00"), ("10:30", "12:30"), ("14:00", "16:00"), ("16:30", "18:00")]
break_time = ("13:00", "14:00")

# Input Date Range
start_date = datetime(2024, 1, 1)
end_date = datetime(2024, 1, 7)

# Helper Functions
def generate_date_range(start, end):
    """Generate a list of dates within a range."""
    return [start + timedelta(days=i) for i in range((end - start).days + 1)]

def assign_venues_and_slots(courses, venues, slots, date_range):
    """Assign venues and slots to courses without conflicts."""
    timetable = []
    used_slots = {}

    for course in courses:
        assigned = False
        for date in date_range:
            for time_slot in slots:
                # Check for slot availability for the same level
                level_key = (date.strftime("%Y-%m-%d"), time_slot, course["level"])
                if level_key in used_slots:
                    continue

                # Find a venue with sufficient capacity
                for venue in venues:
                    if venue["capacity"] >= course["students"]:
                        timetable.append({
                            "date": date.strftime("%Y-%m-%d"),
                            "day": date.strftime("%A"),
                            "time": f"{time_slot[0]} - {time_slot[1]}",
                            "course_code": course["code"],
                            "course_name": course["name"],
                            "program": course["program"],
                            "level": course["level"],
                            "venue": venue["name"],
                            "type": course["type"],
                        })
                        used_slots[level_key] = True
                        assigned = True
                        break
                if assigned:
                    break
            if assigned:
                break
    return timetable

# Main Logic - Continued
date_range = generate_date_range(start_date, end_date)

# Flatten courses into a list
all_courses = []
for college, departments in programs.items():
    for department, courses in departments.items():
        for course in courses:
            course["program"] = department
            all_courses.append(course)

timetable = assign_venues_and_slots(all_courses, venues, exam_slots, date_range)


In [2]:
def assign_invigilators(timetable, invigilators):
    """Assign invigilators randomly to exams."""
    for entry in timetable:
        entry["invigilator"] = random.choice(invigilators)["name"]
    return timetable


In [3]:
def generate_daily_exam_report(timetable):
    """Generate a daily exam report showing all exams scheduled for each day."""
    report = {}
    for entry in timetable:
        date = entry["date"]
        if date not in report:
            report[date] = []
        report[date].append({
            "time": entry["time"],
            "course_code": entry["course_code"],
            "course_name": entry["course_name"],
            "type": entry["type"],
            "venue": entry["venue"],
            "invigilator": entry["invigilator"],
        })
    return report


In [4]:
def generate_departmental_schedule(timetable, department_name):
    """Generate a schedule for a specific department."""
    department_schedule = [
        {
            "date": entry["date"],
            "time": entry["time"],
            "course_code": entry["course_code"],
            "course_name": entry["course_name"],
            "level": entry["level"],
            "venue": entry["venue"],
            "type": entry["type"],
        }
        for entry in timetable
        if entry["program"] == department_name
    ]
    return department_schedule


In [5]:
def export_to_json(data, filename):
    """Export data to a JSON file."""
    with open(filename, "w") as file:
        json.dump(data, file, indent=4)

# Assign Invigilators
timetable_with_invigilators = assign_invigilators(timetable, invigilators)

# Generate Reports
daily_report = generate_daily_exam_report(timetable_with_invigilators)
departmental_schedule = generate_departmental_schedule(timetable_with_invigilators, "Software Engineering")

# Export Reports
export_to_json(timetable_with_invigilators, "timetable.json")
export_to_json(daily_report, "daily_exam_report.json")
export_to_json(departmental_schedule, "software_engineering_schedule.json")
